In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load Dataset
df = pd.read_csv('/content/train (1) (1).csv')

# Separate target and features
y = df['Survived']
X = df.drop('Survived', axis=1)

# Drop irrelevant or high-cardinality columns
X = X.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Define models and hyperparameters
models = {
    "LogisticRegression": {
        "model": LogisticRegression(max_iter=5000),
        "params": {
            "C": [0.01, 0.1, 1, 10],
            "solver": ['liblinear', 'lbfgs']
        }
    },
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [None, 5, 10],
            "min_samples_split": [2, 5]
        }
    },
    "SVC": {
        "model": SVC(),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ['linear', 'rbf'],
            "gamma": ['scale', 'auto']
        }
    }
}

# Cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store results
results = []

# GridSearchCV for each model
for name, mp in models.items():
    print(f"\n🔍 Tuning {name} with GridSearchCV...")
    grid = GridSearchCV(mp["model"], mp["params"], cv=cv, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Best Parameters: {grid.best_params_}")
    print("Classification Report:\n", classification_report(y_test, y_pred))

    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    })

# Convert to DataFrame and display
results_df = pd.DataFrame(results)
print("\n📊 Model Comparison:")
print(results_df.sort_values(by="F1 Score", ascending=False).reset_index(drop=True))



🔍 Tuning LogisticRegression with GridSearchCV...
Best Parameters: {'C': 0.01, 'solver': 'lbfgs'}
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.93      0.85       110
           1       0.84      0.59      0.69        69

    accuracy                           0.80       179
   macro avg       0.81      0.76      0.77       179
weighted avg       0.80      0.80      0.79       179


🔍 Tuning RandomForest with GridSearchCV...
Best Parameters: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200}
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.93      0.85       110
           1       0.84      0.61      0.71        69

    accuracy                           0.80       179
   macro avg       0.82      0.77      0.78       179
weighted avg       0.81      0.80      0.80       179


🔍 Tuning SVC with GridSearchCV...
Best Parameters: {'C': 1, 'gamma': 'scale'